In [ ]:
from collections import defaultdict

class LFUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.data = {}  # key -> value
        self.freq = {}  # key -> frequency
        self.groups = defaultdict(list)  # frequency -> list of keys
        self.min_freq = 0  # minimum frequency in the cache

    def get(self, key):
        if key not in self.data:
            return -1
        # Increase frequency
        self._update_frequency(key)
        return self.data[key]

    def put(self, key, value):
        if self.capacity == 0:
            return

        if key in self.data:
            # Update value and increase frequency
            self.data[key] = value
            self._update_frequency(key)
        else:
            if len(self.data) >= self.capacity:
                # Remove the least frequently used element
                self._evict_least_frequent()
            # Add new element
            self.data[key] = value
            self.freq[key] = 1
            self.groups[1].append(key)
            self.min_freq = 1

    def _update_frequency(self, key):
        # Current frequency
        current_freq = self.freq[key]
        # Update frequency
        self.freq[key] += 1
        # Remove key from the old group
        self.groups[current_freq].remove(key)
        if not self.groups[current_freq]:
            del self.groups[current_freq]
            if current_freq == self.min_freq:
                self.min_freq += 1
        # Add key to the new group
        self.groups[self.freq[key]].append(key)

    def _evict_least_frequent(self):
        # Remove the item with the minimum frequency
        least_frequent_keys = self.groups[self.min_freq]
        oldest_key = least_frequent_keys.pop(0)
        if not least_frequent_keys:
            del self.groups[self.min_freq]
        del self.data[oldest_key]
        del self.freq[oldest_key]


In [ ]:
if __name__ == "__main__":
    cache = LFUCache(3)

    cache.put(1, "Banana")
    cache.put(2, "Pear")
    cache.put(3, "Apple")
    
    print(cache.get(1))  
    
    cache.put(4, "Melon")
    
    print(cache.get(2))  # will output -1 (not found)